# Absurdle Solver
---

[Absurdle](https://absurdle.online/) is an online word puzzle game that closely resembles the massively popular [Wordle](https://www.nytimes.com/games/wordle/index.html), implementing several of its core mechanics in the pursuit of a user correctly determining an unseen 5-letter word pattern through the repeated testing of several test patterns (guesses) to determine common letters and letter placements. The objective of a puzzle is to determine the unseen word with as few test patterns as possible.

The key difference with Absurdle is its __adversarial__ nature. As opposed to a fixed hidden pattern which is revealed through several test patterns, Absurdle dynamically determines the target word based on test patterns. It does so by maximising the entropy of the search: at each guess, the partition containing the 'target' word is chosen to be the largest collection of words sharing a matching pattern with the provided guess. This continues until the partition contains precisely one word, the target word. More details on the game's mechanics can be found in the [Absurdle Blog](https://qntm.org/absurdle).

While convergence to a target word is guaranteed, the reduction in the search space can be slow based on the user's sequence of guesses. As a result, Absurdle requires noticeably more turns to derive a solution than Wordle. While a seasoned Wordle player can often pick the next move which sufficiently minimises entropy, an Absurdle player is locked into pursuing the maximal entropic sequence based on their word choices.

This workbook employs search strategies to determine an optimal Absurdle playing: that is, the shortest sequence of guesses required to end the game.

In [124]:
# packages
import tensorflow as tf
import pandas as pd
import kagglehub as kh

### Valid Wordle Guess/Answer Datasets

Here we download the publicly available list of 5-letter Wordle guesses and answers. It is important to note that Wordle allows a sizeable superset of valid answers to be treated as valid guesses, i.e. `AAHED` is a valid _guess_ but not a valid _answer_ when playing Wordle. We make the simplifying assumption that the Absurdle vocabulary closely mirrors that of valid Wordle guesses and answers and an optimal play can be derived from the Wordle vocabulary.

In [125]:
dataset_path = kh.dataset_download("lucashohmann/wordle-valid-guesses-and-answers")
valid_answers = pd.read_csv(f'{dataset_path}/valid_answers.csv')
valid_guesses = pd.read_csv(f'{dataset_path}/valid_guesses.csv')

n_guesses, _ = valid_guesses.shape
n_answers, _ = valid_answers.shape
word_length = 5

In [126]:
valid_guesses

,word
0,aahed
1,aalii
2,aargh
3,aarti
4,abaca
...,...
10660,zuzim
10661,zygal
10662,zygon
10663,zymes


In [127]:
valid_answers

,word
0,aback
1,abase
2,abate
3,abbey
4,abbot
...,...
2304,young
2305,youth
2306,zebra
2307,zesty


### Wordle Pattern Matching Algorithm

Here we re-implement the standard Wordle rules for comparing a test pattern against a predefined target pattern.
The pattern matching logic is as follows:
1. Letters in `test` which occur in the same position in `target` are marked as GREEN (**2**).
2. Failing which, those letters in `test` and `target` which do not match in position but which still appear in both words are iteratively marked off from left to right in `test` as YELLOW (**1**), removing the letter from consideration in the `target`.
3. Failing which, remaining letters in `test` are marked in BLACK (**0**).

Some examples of this may feel counter-intuitive but are worth considering.
Suppose `test='SPEED'` and `target='ABIDE'`:
* rule 1 indicates no letters in `test` are marked as **2** as no letters occur in the same position in both words.
* rule 2 indicates the 'E' in index 2 of `test` be marked **1** as it also appears in index 4 of `target`. This 'E' in `target` cannot be reused for further consideration. The same logic applies to the 'D' in index 4 of `test`.
* rule 3 indicates that all remaining letters be marked **0**.

The resulting pattern is **[0,0,1,0,1]**

As all generated matching patterns can be mapped to ternary strings with B=0, Y=1, G=2, we enumerate patterns with the decimal value of the ternary string. e.g. the above pattern results in pattern no. **10**. This enumeration does not reflect the strength of the matched pattern.

In [128]:
# Helper function to append the sequence number of a particular character in a word, e.g. the first 'e', second 'a', etc.
@tf.function
def append_sequence_no(word):
    counter = tf.cast(tf.scan(lambda a, x: a + tf.one_hot(x, 26), word, tf.zeros(26)), tf.int32)
    sequence_no = tf.gather_nd(counter, tf.transpose(tf.stack((tf.range(word_length), word))))
    return tf.transpose(tf.stack((word, sequence_no)))

# Wordle pattern matching algorithm
@tf.function
def match_pattern_vec(test, target):

    # Turn words into offset byte vectors
    test =  tf.reshape((tf.strings.unicode_decode(test,'UTF-8') - 97).to_tensor(-1),(word_length,))
    target =  tf.reshape((tf.strings.unicode_decode(target,'UTF-8') - 97).to_tensor(-1),(word_length,))

    # Form green mask by comparing content in exact placement
    green_mask = tf.cast(test == target, tf.int32)

    # Mask the test word by removing those characters covered by the green mask
    test_masked = tf.where(tf.cast(green_mask,tf.bool), -1, test)

    # Determine sequence numbers for the test and target
    test_sequenced = append_sequence_no(test_masked)
    target_sequenced = append_sequence_no(target)

    # Form yellow mask by looking for matching character and sequence number in any order
    yellow_mask = tf.map_fn(lambda t: tf.cast(tf.reduce_any(tf.reduce_all(t == target_sequenced,axis=-1)), tf.int32), test_sequenced)

    # Return ternary array
    return (2 * green_mask) + (1 * yellow_mask)

@tf.function()
def match_pattern(test, target):
    return tf.reduce_sum(match_pattern_vec(test, target) * 3 ** tf.range(word_length))

In [129]:
@tf.function
def reduce_targets_by_test(targets, test):
    patterns = tf.map_fn(lambda target: match_pattern(test, target),targets,fn_output_signature=tf.int32)
    unique_patterns, pattern_indices, pattern_counts = tf.unique_with_counts(patterns)
    idx_max = tf.cast(tf.argmax(pattern_counts), tf.int32)
    return targets[pattern_indices == idx_max]

In [130]:
reduce_targets_by_test(valid_answers, valid_guesses.iloc[0])

<tf.Tensor: shape=(448, 1), dtype=string, numpy=
array([[b'bigot'],
       [b'billy'],
       [b'bingo'],
       [b'bison'],
       [b'bitty'],
       [b'blimp'],
       [b'blink'],
       [b'bliss'],
       [b'blitz'],
       [b'block'],
       [b'bloom'],
       [b'blown'],
       [b'bluff'],
       [b'blunt'],
       [b'blurb'],
       [b'blurt'],
       [b'bobby'],
       [b'bongo'],
       [b'bonus'],
       [b'booby'],
       [b'boost'],
       [b'booty'],
       [b'boozy'],
       [b'bosom'],
       [b'bossy'],
       [b'brick'],
       [b'bring'],
       [b'brink'],
       [b'briny'],
       [b'brisk'],
       [b'broil'],
       [b'brook'],
       [b'broom'],
       [b'brown'],
       [b'brunt'],
       [b'buggy'],
       [b'built'],
       [b'bulky'],
       [b'bully'],
       [b'bunny'],
       [b'burly'],
       [b'burnt'],
       [b'burst'],
       [b'buxom'],
       [b'civic'],
       [b'civil'],
       [b'click'],
       [b'cliff'],
       [b'climb'],
       [b'cling'],
 